In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
from neural_map import NeuralMap, _plot
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv("datasets/ANCESTRY_LACRN.tsv", sep="\t")
l_columns = ['IndID', 'SentrixID', 'Sex', 'Source', 'Region', 'Population', 'Ancestry']
col = 'Ancestry'
data = df.drop(columns=l_columns).values
data.shape

In [ ]:
som = NeuralMap(z=data.shape[1], x=6, y=6, metric='correlation')

In [ ]:
som.train(data=data, verbosity=True, eval_data=data, num_epochs=30) # , radius_decay_function='exponential'

In [ ]:
som.plot_analysis(data, min_cluster_size=3, show_empty_nodes=True)

In [ ]:
labels = df[col].values
types = df[col].unique()

for i in range(3):
    som.plot_analysis(data, min_cluster_size=3, show_quantization_error=False, show_empty_nodes=False, labels=labels, types_to_display=types[i * 3 : (i + 1) * 3])

In [ ]:
for i in range(som.z):
    som.plot_analysis(data, min_cluster_size=3, show_quantization_error=False, show_empty_nodes=False, attached_values=data[:, i])

In [ ]:
som.plot_u_matrix(detailed=True, borders=True)

In [ ]:
clusters = 4

labels, centers = som.k_means(clusters)
_plot.tiles(som.cart_coord, som.hexagonal, labels, norm=False, title='Clustering con K-means. Valor de K = ' + str(clusters), labels=list(range(clusters)))

In [ ]:
som.plot_set_weight_vectors(cluster=0)
som.plot_set_weight_vectors(cluster=1)

In [ ]:
from BootstrapCCpy import BootstrapCCpy
from sklearn.cluster import KMeans

In [ ]:
K_CLUSTERS = 5
B_SAMPLINGS = 36

bcc = BootstrapCCpy(cluster=KMeans().__class__, K=K_CLUSTERS, B=B_SAMPLINGS)
bcc.fit(som.weights.reshape((-1, som.z)), verbose=True)
bcc.get_best_k()
bcc.plot_consensus_distribution()
bcc.plot_consensus_heatmap()